In [ ]:
import pandas as pd
from tqdm.notebook import tqdm_notebook
from dateutil import parser
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import numpy as np
import joblib

import warnings
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
from textblob import TextBlob
import emoji
import itertools 
import matplotlib.style as style
style.use('ggplot')
warnings.filterwarnings('ignore')

In [ ]:
# Read txt files
f = open("groupchat_backup.txt","r", encoding="utf8")
content = f.read()
print(content)

In [ ]:
# Memisahkan konten menjadi per line
lines = []
count = 0
line = ""
for letter in content:#
    if letter == '\n':
        lines.append(line)
        line = ""
    else:
        line = line + letter
        
lines.remove('\ufeff')

In [ ]:
from dateutil.parser import parse

def isDateUniversal(string, fuzzy=False):
    """
    Return whether the string can be interpreted as a date.

    :param string: str, string to check for date
    :param fuzzy: bool, ignore unknown tokens in string if True
    """
    try: 
        parse(string, fuzzy=fuzzy)
        return True

    except ValueError:
        return False

In [ ]:
from datetime import datetime

def isDate(date_text):
    try:
        if date_text != datetime.strptime(date_text, "%m/%d/%Y").strftime("%m/%d/%Y"):
            raise ValueError
        return True
    except ValueError:
        return False

In [ ]:
from datetime import datetime

def isTime(time_text):
    try:
        if time_text != datetime.strptime(time_text, "%H:%M").strftime("%H:%M"):
            raise ValueError
        return True
    except ValueError:
        return False

In [ ]:
def dateConvertStr(text):
    if len(text) == 8:
        text = text[:0] + '0' + text[0:]
        text = text[:3] + '0' + text[3:]
    elif len(text) == 9:
        if text[1] == '/':
            text = '0' + text[0:]
        else:
            text = text[:3] + '0' + text[3:]
    return text

In [ ]:
# Inisialisasi Dataframe
dict = {
    'date':[],
    'time':[],
    'name':[],
    'chat':''
}

df = pd.DataFrame(dict)

start_time = datetime.now()
# Preprocessing Test
time = ""
name = ""
chat = ""
date = ""


for line in lines:
    # Split per line
    words = []
    words = line.split('\t')
    print(words)
    if(words == ['']):
        continue
    elif (len(words) == 1):
        if not(isDateUniversal(words[0])):
            # Add new line to chat
            chat = chat + " \n" +words[0]
            # Remove last row
            df = df[:-1]
            # Add updated row to dataframe
            df.loc[len(df.index)] = [date, time, name, chat]
        elif isDateUniversal (words[0]):
            # Converting
            word = words[0]
            # Cut nama hari
            date_potong_str = word[5:-1] + word[-1]
            # Convert ke format mm/dd/yyyy
            date_coba = dateConvertStr(date_potong_str)
            if isDate(date_coba):
                date = date_coba
    elif (len(words) == 2):
        name = ""
        time, chat = words[0], words[1]
        
        # Next Processing 
#         actions = ['joined the group.',
#                    'left the group.',
#                   'invited']
#         chats = chat.split(' ')
#         subchat = chats[-3] + ' ' + chats[-2] + ' ' + chats[-1]
#         print(subchat)
#         if (subchat =|= actions[0]):
#             name = 'someone' # ubah jadi chat dikurang subchat
# #             name = chat.remove(subchat)
#             chat = subchat       
        # Add row to dataframe
        df.loc[len(df.index)] = [date, time, name, chat]
    elif (len(words) == 3):
        time, name, chat = words[0], words[1], words[2]
        # Add row to dataframe
        df.loc[len(df.index)] = [date, time, name, chat]

print("Finished at " + str(datetime.now() - start_time))

In [ ]:
# Menambah kolom istime sebagai boolean apakah format time benar atau salah
df['istime'] = [True if isTime(time) else False for time in df['time']]
# Drop kolom dengan format time salah
idx_drop = df.index[[df['istime'] == False]]
df.drop(idx_drop, inplace = True)

In [ ]:
date = pd.Series(df['date'])
time = pd.Series(df['time'])
df['datetime'] = pd.to_datetime(date.str.cat(time, sep=' '))

In [ ]:
# Drop kolom tidak terpakai
df.drop(columns=['istime','date','time'], inplace = True)

In [ ]:
# Rearrange urutan kolom
df = df[['datetime','name','chat']]